# Extracting triplets from data for further analysis

Graph data can be stored in two main forms:
+ LPS: labeled property graph with no schema and arbitary label properties
+ RDF: with predefined types of relations; supports resoning 

### How to automatically fill this data.
For example, we can extract Named Entities, then, from natural text, using sematic properties of language, we can extend our data. 

## Language semantics

As a baseline in extracting relations from text we can use spaCy capabilities of semantic parsing.

In [12]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 11.8 MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047106 sha256=c942e3487e851c002aa5fd481e4740035e51264d665a4a8f8bdc60fc81174ed3
  Stored in directory: /tmp/pip-ephem-wheel-cache-wqqzkxpm/wheels/ee/4d/f7/563214122be1540b5f9197b52cb3ddb9c4a8070808b22d5a84
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [1]:
import spacy

nlp = spacy.load("en_core_web_sm")
text = "Demand for lithium is expected to outpace global supply as consumers switch to battery-powered vehicles. With China currently leading in processing of the vital raw material, the U.S. government is looking to boost domestic production."
doc = nlp(text)

In [3]:
for token in doc:
    print(f"{token.text}:  {token.pos_}")

Demand:  NOUN
for:  ADP
lithium:  NOUN
is:  AUX
expected:  VERB
to:  PART
outpace:  VERB
global:  ADJ
supply:  NOUN
as:  SCONJ
consumers:  NOUN
switch:  VERB
to:  ADP
battery:  NOUN
-:  PUNCT
powered:  VERB
vehicles:  NOUN
.:  PUNCT
With:  ADP
China:  PROPN
currently:  ADV
leading:  VERB
in:  ADP
processing:  NOUN
of:  ADP
the:  DET
vital:  ADJ
raw:  ADJ
material:  NOUN
,:  PUNCT
the:  DET
U.S.:  PROPN
government:  NOUN
is:  AUX
looking:  VERB
to:  PART
boost:  VERB
domestic:  ADJ
production:  NOUN
.:  PUNCT


In [6]:
for entity in doc.ents:
    print(entity.text, entity.start_char, entity.end_char, entity.label_)

China 110 115 GPE
U.S. 179 183 GPE


Spacy provides us with a dependency tree of some kind

In [8]:
import spacy
from nltk import Tree

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_


for sent in doc.sents:
    print(sent.text)
    to_nltk_tree(sent.root).pretty_print()

Demand for lithium is expected to outpace global supply as consumers switch to battery-powered vehicles.
                    expected                                       
  _____________________|________                                    
 |   |     |                 outpace                               
 |   |     |      ______________|________                           
 |   |     |     |     |               switch                      
 |   |     |     |     |         ________|________________          
 |   |     |     |     |        |        |                to       
 |   |     |     |     |        |        |                |         
 |   |   Demand  |     |        |        |             vehicles    
 |   |     |     |     |        |        |                |         
 |   |    for    |   supply     |        |             powered     
 |   |     |     |     |        |        |         _______|______   
 is  .  lithium  to  global     as   consumers battery           - 

With

With help of matchers we can extract some relations

## Temporal information

Along with temporal (datetime) data found in metadata, we can extract some of it

In [9]:
!pip install dateparser

     |████████████████████████████████| 279 kB 2.2 MB/s eta 0:00:01


In [15]:
import dateparser

text = "The analysts expects AMD fiscal earnings of $2.04 a share in 2021, $2.59 a share in 2022, and $2.90 a share in 2023, while analysts surveyed by FactSet expect per-share earnings of $1.95, $2.51, and $3.23, respectively. "

dateparser.parse(text)

In [18]:
dateparser.parse("In December")

datetime.datetime(2021, 12, 26, 0, 0)

works poorly for my purposes. :(